**RealESRGAN-GColab**
==
**RealESRGAN-GColab** leverages the power of **Nvidia T4 GPU** available in **Google Colab** to **enhance image quality with sharper details and improved clarity automatically**. By utilizing the Nvidia T4 GPU, **Remini-AI** is capable of delivering superior and **faster results** in enhancing image quality within the **Google Colab** environment.

In [ ]:
#@title **Install Package**

!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

from IPython.display import clear_output
clear_output()
print("Success To Install package")

In [ ]:
#@title **Create Remini-AI Model With RealESRGAN**
from RealESRGAN import RealESRGAN
from IPython.display import clear_output
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

try:
    model = RealESRGAN(device, scale=int(model_scale))
    model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')
    clear_output()
    print(f"Model Scale {model_scale} Has Been Installed!")
except Exception as e:
    clear_output()
    print('Something Is Wrong..')
    print(e)

In [ ]:
#@title **Upload And Get Result**
import os
import requests
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile
from PIL import Image
from IPython.display import clear_output
import numpy as np

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Finished! Image saved to {result_image_path}')
        print('Uploading Photo To filezone.my.id..')

        # Upload the processed image to the specified URL
        url = 'https://filezone.my.id/upload'
        with open(result_image_path, 'rb') as file:
            files = {'file': file}
            response = requests.post(url, files=files)
            response_json = response.json()
            if response.status_code == 200:
                clear_output()
                print('Output Result')
                print('')
                print('Filename:', response_json['result']['originalname'])
                print('File Size:', response_json['result']['filesize'])
                print('URL File:', response_json['result']['url_file'])
            else:
                clear_output()
                print('Upload failed. Response:', response.text)

        # Remove the original and processed files
        os.remove(filename)
        os.remove(result_image_path)

# Upload files
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        print('Processing:', filename)
        process_input(filename)
except KeyboardInterrupt:
    print('Canceled upload.')

# Remove the upload_folder and result_folder directories
shutil.rmtree(upload_folder)
shutil.rmtree(result_folder)

**Credits✨**
==
Script by **RizzyFuzz**

Powered By **RealESRGAN**

**CaliphDev** (File Uploader API Provider)